In [2]:
try:
    import pandas as pd
    import joblib
    import matplotlib.pyplot as plt
    %matplotlib inline
    import numpy as np
    import pandas as pd
    import seaborn as sns
    import numpy as np
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.callbacks import EarlyStopping
    from xgboost import XGBClassifier
    from scipy import stats
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    from sklearn.datasets import make_regression, make_swiss_roll
    from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.svm import SVC, SVR
    from sklearn.tree import DecisionTreeClassifier

    print('All libraries imported successfully!')
except ImportError:
    print('Some libraries failed to import.')

/Users/beau/anaconda3/envs/dev/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2024-07-24 09:26:37.925294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


All libraries imported successfully!


In [3]:
# Load the dataset
data = pd.read_csv('/Users/beau/Desktop/house_of_hope/clean_df.csv')
data

,EDUC,MARSTAT,SERVICES,LOS,PSOURCE,NOPRIOR,ARRESTS,EMPLOY,METHUSE,PSYPROB,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,3,1,7,7,6,1,0,2,2,1,...,0,0,0,0,0,0,9,4,0,1
1,3,4,7,8,1,1,0,2,2,1,...,0,0,0,0,0,1,9,4,0,3
2,3,4,7,7,3,1,0,1,2,2,...,0,0,0,0,0,0,9,4,0,2
3,5,1,7,4,7,0,0,1,2,2,...,0,0,0,0,0,0,9,4,0,1
4,3,4,7,3,7,1,1,4,2,1,...,0,0,0,0,0,0,9,4,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614991,5,1,7,2,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3
2614992,5,1,7,1,3,1,0,3,2,2,...,0,0,0,0,0,0,5,3,1,3
2614993,5,1,7,1,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3
2614994,5,1,7,1,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3


In [4]:
# what columns have -9 values?
data.columns[data.isin([-9]).any()].tolist() # This line of code will return the columns that have -9 values

# if the column has -9 values, display the column name and the number of -9 values
for col in data.columns: # This loop will iterate through the columns
    if data[col].isin([-9]).any(): # This condition will check if the column has -9 values
        print(f'{col}: {data[col].isin([-9]).sum()}') # This line of code will display the column name and the number of -9 values

In [6]:
# Display the HLTHINS column
data['HLTHINS']

0          2
1          2
2          4
3          3
4          2
          ..
2614991    4
2614992    4
2614993    4
2614994    4
2614995    4
Name: HLTHINS, Length: 2614996, dtype: int64

In [7]:
# Show the rows with the -9 values in the HLTHINS column
display(data[data['HLTHINS'] == -9]) # This line of code will display the rows with -9 values in the HLTHINS column

,EDUC,MARSTAT,SERVICES,LOS,PSOURCE,NOPRIOR,ARRESTS,EMPLOY,METHUSE,PSYPROB,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG


In [8]:
# In the HLTHINS column, replace all -9 values with 4
data['HLTHINS'] = data['HLTHINS'].replace(-9, 4) # This line of code will replace all -9 values with 4 in the HLTHINS column

In [9]:
# Check to see if the HLTHINS column still has -9 values. If it does, display the column name and the number of -9 values, if not, display 'No -9 values found'
if data['HLTHINS'].isin([-9]).any(): # This condition will check if the HLTHINS column has -9 values
    print(f'HLTHINS: {data["HLTHINS"].isin([-9]).sum()}') # This line of code will display the column name and the number of -9 values
else:
    print('No -9 values found') # This line of code will display 'No -9 values found'


No -9 values found


In [10]:
# Create a copy of the dataset with -9 values in the HLTHINS column replaced with 4
df1 = data.copy() # This line of code will create a copy of the dataset with -9 values in the HLTHINS column replaced with 4

In [11]:
df1 # This line of code will display the new dataset

,EDUC,MARSTAT,SERVICES,LOS,PSOURCE,NOPRIOR,ARRESTS,EMPLOY,METHUSE,PSYPROB,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,3,1,7,7,6,1,0,2,2,1,...,0,0,0,0,0,0,9,4,0,1
1,3,4,7,8,1,1,0,2,2,1,...,0,0,0,0,0,1,9,4,0,3
2,3,4,7,7,3,1,0,1,2,2,...,0,0,0,0,0,0,9,4,0,2
3,5,1,7,4,7,0,0,1,2,2,...,0,0,0,0,0,0,9,4,0,1
4,3,4,7,3,7,1,1,4,2,1,...,0,0,0,0,0,0,9,4,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614991,5,1,7,2,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3
2614992,5,1,7,1,3,1,0,3,2,2,...,0,0,0,0,0,0,5,3,1,3
2614993,5,1,7,1,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3
2614994,5,1,7,1,3,1,0,3,1,2,...,0,0,0,0,0,0,5,3,1,3


In [12]:
# Are there any columns with -9 values. If the column has -9 values, display the column name, if not, display 'No -9 values found'
for col in df1.columns: # This loop will iterate through the columns
    if df1[col].isin([-9]).any(): # This condition will check if the column has -9 values
        print(f'{col}') # This line of code will display the column name
    else:
        print('No -9 values found') # This line of code will display 'No -9 values found'

No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values found
No -9 values

## Filters and Correlation

### FREQ1: Frequency of use at admission (primary)
Specifies the frequency of use of the corresponding substance identified in Substance Use (SUB1) at admission.

1 = No use in the past month

2 = Some use

3 = Daily use

In [13]:
df_no_use_past_month = df1.loc[df1["FREQ1"] == 1]
reason = df_no_use_past_month["REASON"]

In [15]:
df_corr = df_no_use_past_month.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

SUB1_D                   0.054761
EMPLOY_D                 0.060613
SUB2                     0.067987
SUB2_D                   0.075338
METHUSE                  0.080042
PRIMPAY                  0.086728
PSYPROB                  0.105745
FREQ_ATND_SELF_HELP_D    0.118209
DIVISION                 0.193650
REGION                   0.204888
REASON                   1.000000
FREQ1                         NaN
Name: REASON, dtype: float64

In [17]:
df_some_use = df1.loc[df1["FREQ1"] == 2] 
df_some_use["REASON"].value_counts() 

REASON
1    445189
0    267133
Name: count, dtype: int64

In [18]:
df_corr = df_some_use.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

FREQ_ATND_SELF_HELP_D    0.063935
ALCFLG                   0.070726
AGE                      0.074216
SUB2_D                   0.077306
METHUSE                  0.084743
SUB1_D                   0.096061
SUB1                     0.102374
DIVISION                 0.110964
REGION                   0.119862
EMPLOY_D                 0.183763
REASON                   1.000000
FREQ1                         NaN
Name: REASON, dtype: float64

In [23]:
df_daily_use = df1.loc[df1["FREQ1"] == 3] 
df_daily_use["REASON"].value_counts() 

REASON
1    623723
0    370654
Name: count, dtype: int64

In [24]:
df_corr = df_daily_use.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

FREQ1_D                  0.047661
SUB2_D                   0.053994
DIVISION                 0.068227
AGE                      0.074637
REGION                   0.076076
METHUSE                  0.088909
FREQ_ATND_SELF_HELP_D    0.095793
ALCFLG                   0.106234
SUB1_D                   0.108989
SUB1                     0.128215
REASON                   1.000000
FREQ1                         NaN
Name: REASON, dtype: float64

### FREQ2: Frequency of use at admission (secondary)
Specifies the frequency of use of the corresponding substance identified in Substance Use (SUB2) at admission.

1 = No use in the past month

2 = Some use

3 = Daily use

In [25]:
df_no_use_past_month = df1.loc[df1["FREQ2"] == 1]
df_no_use_past_month["REASON"].value_counts() 

REASON
1    1071874
0     611295
Name: count, dtype: int64

In [ ]:
df_corr = df_no_use_past_month.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

SUB3_D                   0.060859
SUB2                     0.067357
FREQ_ATND_SELF_HELP_D    0.068705
AGE                      0.070242
SUB1                     0.083600
SUB1_D                   0.085149
METHUSE                  0.089532
SUB2_D                   0.098701
DIVISION                 0.134174
REGION                   0.154860
REASON                   1.000000
Name: REASON, dtype: float64

In [26]:
df_some_use = df1.loc[df1["FREQ2"] == 2]
df_some_use["REASON"].value_counts() 

REASON
1    315012
0    200003
Name: count, dtype: int64

In [27]:
df_corr = df_some_use.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

SUB1_D                   0.050802
PSYPROB                  0.056240
FREQ_ATND_SELF_HELP      0.063714
SUB1                     0.066406
FREQ1_D                  0.076899
METHUSE                  0.085240
FREQ_ATND_SELF_HELP_D    0.118578
DIVISION                 0.134849
REGION                   0.147079
EMPLOY_D                 0.162806
REASON                   1.000000
FREQ2                         NaN
Name: REASON, dtype: float64

In [28]:
df_daily_use = df1.loc[df1["FREQ2"] == 3]
df_daily_use["REASON"].value_counts()

REASON
1    264408
0    152404
Name: count, dtype: int64

In [29]:
df_corr = df_daily_use.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

OTHERFLG                 0.059018
AGE                      0.063761
ALCDRUG                  0.065774
ALCFLG                   0.066353
SUB1_D                   0.072476
METHUSE                  0.081130
SUB1                     0.086333
FREQ_ATND_SELF_HELP_D    0.098139
DIVISION                 0.105186
REGION                   0.110646
REASON                   1.000000
FREQ2                         NaN
Name: REASON, dtype: float64

### IDU: Current IV drug use reported at admission

0 = IDU not reported

1 = IDU reported

In [30]:
df_not_reported = df1.loc[df1["IDU"] == 0]
df_not_reported["REASON"].value_counts()

REASON
1    1266282
0     707035
Name: count, dtype: int64

In [31]:
df_corr = df_not_reported.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

ALCFLG                   0.057671
SUB2_D                   0.074199
METHUSE                  0.076254
AGE                      0.079163
EMPLOY_D                 0.081925
SUB1_D                   0.091121
FREQ_ATND_SELF_HELP_D    0.095314
SUB1                     0.096407
DIVISION                 0.121981
REGION                   0.131716
REASON                   1.000000
IDU                           NaN
Name: REASON, dtype: float64

In [32]:
df_reported = df1.loc[df1["IDU"] == 1]
df_reported["REASON"].value_counts()

REASON
1    385012
0    256667
Name: count, dtype: int64

In [33]:
df_corr = df_reported.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

OTHERFLG                 0.041830
FREQ_ATND_SELF_HELP      0.045129
PRIMPAY                  0.045526
FREQ1_D                  0.048878
MTHAMFLG                 0.067204
EMPLOY_D                 0.094685
METHUSE                  0.099586
FREQ_ATND_SELF_HELP_D    0.102537
DIVISION                 0.114337
REGION                   0.122407
REASON                   1.000000
IDU                           NaN
Name: REASON, dtype: float64

### MARFLG: Marijuana/hashish reported at admission

0 = Substance not reported

1 = Substance reported

In [34]:
df_sub_not_reported = df1.loc[df1["MARFLG"] == 0]
df_sub_not_reported["REASON"].value_counts()

REASON
1    1164083
0     618376
Name: count, dtype: int64

In [35]:
df_corr = df_sub_not_reported.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

PRIMPAY                  0.059873
ALCFLG                   0.071890
EMPLOY_D                 0.074538
FREQ_ATND_SELF_HELP_D    0.083762
SUB1_D                   0.088043
SUB2_D                   0.090820
SUB1                     0.094113
METHUSE                  0.102799
DIVISION                 0.116416
REGION                   0.125374
REASON                   1.000000
MARFLG                        NaN
Name: REASON, dtype: float64

In [36]:
df_sub_reported = df1.loc[df1["MARFLG"] == 1]
df_sub_reported["REASON"].value_counts()

REASON
1    487211
0    345326
Name: count, dtype: int64

In [37]:
df_corr = df_sub_reported.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

LIVARAG_D                0.053981
SUB1_D                   0.057026
SUB1                     0.057266
FREQ_ATND_SELF_HELP      0.057304
FREQ2_D                  0.060662
METHUSE                  0.060743
EMPLOY_D                 0.102530
FREQ_ATND_SELF_HELP_D    0.109749
DIVISION                 0.138154
REGION                   0.145461
REASON                   1.000000
MARFLG                        NaN
Name: REASON, dtype: float64

### ALCDRUG: Substance use type

0 = None

1 = Alcohol only

2 = Other drugs only

3 = Alcohol and other drugs

In [40]:
df_none = df1.loc[df1["ALCDRUG"] == 0]
df_none["REASON"].value_counts()

REASON
1    79181
0    29828
Name: count, dtype: int64

In [41]:
df_corr = df_none.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

REASON      1.0
METHFLG     NaN
PCPFLG      NaN
HALLFLG     NaN
AMPHFLG     NaN
STIMFLG     NaN
TRNQFLG     NaN
BARBFLG     NaN
SEDHPFLG    NaN
INHFLG      NaN
OTCFLG      NaN
ALCDRUG     NaN
Name: REASON, dtype: float64

In [42]:
df_alcohol_only = df1.loc[df1["ALCDRUG"] == 1]
df_alcohol_only["REASON"].value_counts()

REASON
1    314328
0    122676
Name: count, dtype: int64

In [43]:
df_corr = df_alcohol_only.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

HALLFLG    NaN
MTHAMFLG   NaN
AMPHFLG    NaN
STIMFLG    NaN
BENZFLG    NaN
TRNQFLG    NaN
BARBFLG    NaN
SEDHPFLG   NaN
INHFLG     NaN
OTCFLG     NaN
OTHERFLG   NaN
ALCDRUG    NaN
Name: REASON, dtype: float64

In [44]:
df_other_drugs_only = df1.loc[df1["ALCDRUG"] == 2]
df_other_drugs_only["REASON"].value_counts()

REASON
1    816757
0    549849
Name: count, dtype: int64

In [45]:
df_corr = df_other_drugs_only.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

FREQ_ATND_SELF_HELP      0.053437
SUB2_D                   0.056182
LIVARAG_D                0.056994
MTHAMFLG                 0.071407
METHUSE                  0.091384
FREQ_ATND_SELF_HELP_D    0.101173
DIVISION                 0.125313
EMPLOY_D                 0.128880
REGION                   0.132602
REASON                   1.000000
ALCFLG                        NaN
ALCDRUG                       NaN
Name: REASON, dtype: float64

In [46]:
df_alchohol_and_other_drugs = df1.loc[df1["ALCDRUG"] == 3]
df_alchohol_and_other_drugs["REASON"].value_counts()

REASON
1    441028
0    261349
Name: count, dtype: int64

In [47]:
df_corr = df_alchohol_and_other_drugs.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

FREQ1                    0.051701
METHUSE                  0.055244
AGE                      0.070446
EMPLOY_D                 0.074177
SUB1_D                   0.074782
SUB1                     0.079235
FREQ_ATND_SELF_HELP_D    0.119491
DIVISION                 0.124381
REGION                   0.129853
REASON                   1.000000
ALCFLG                        NaN
ALCDRUG                       NaN
Name: REASON, dtype: float64

### PSYPROB: Co-occurring mental and substance use disorders

1 = Yes

2 = No

In [48]:
df_yes = df1.loc[df1["PSYPROB"] == 1]
df_yes["REASON"].value_counts()

REASON
1    742589
0    483927
Name: count, dtype: int64

In [49]:
df_corr = df_yes.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

FREQ1                    0.037745
PRIMPAY                  0.049226
ALCFLG                   0.065345
AGE                      0.069312
METHUSE                  0.080976
DIVISION                 0.088218
REGION                   0.090103
SUB1                     0.094036
SUB1_D                   0.094944
FREQ_ATND_SELF_HELP_D    0.112967
REASON                   1.000000
PSYPROB                       NaN
Name: REASON, dtype: float64

In [50]:
df_no = df1.loc[df1["PSYPROB"] == 2]
df_no["REASON"].value_counts()

REASON
1    908705
0    479775
Name: count, dtype: int64

In [51]:
df_corr = df_no.corr()
df_corr.unstack().sort_values()
variable = df_corr["REASON"].sort_values()
variable.tail(12)

LIVARAG_D                0.066300
AGE                      0.067936
SUB1_D                   0.079170
FREQ_ATND_SELF_HELP_D    0.081433
METHUSE                  0.088408
SUB1                     0.095644
SUB2_D                   0.109432
DIVISION                 0.136680
EMPLOY_D                 0.150714
REGION                   0.153591
REASON                   1.000000
PSYPROB                       NaN
Name: REASON, dtype: float64